In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import paraguayfloodspy.visualize as viz # custom plotting library

Define parameters

In [ ]:
%matplotlib inline
savefig = False

Read in data

In [ ]:
wt = pd.read_csv('../data/derived/WeatherTypes.csv', parse_dates=True, index_col='time')
mjo = pd.read_csv('../data/accessed/daily_indices.csv', parse_dates=True, index_col='time')
enso = pd.read_csv('../data/accessed/monthly_indices.csv', parse_dates=True, index_col='time')
enso = enso.resample('1D').ffill()
mrg = wt.join(mjo).join(enso)
mrg.loc[mrg['amplitude'] < 1, 'phase'] =  0
mrg.head()

Fit a multinomial regression

In [ ]:
from sklearn.linear_model import LogisticRegression
y_train = mrg[['wtype']].values.ravel()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder().fit_transform(mrg['phase'].values[:, np.newaxis]).toarray()
X_train = np.append(enc, mrg['nino_34'].values[:, np.newaxis], axis=1)
fit = LogisticRegression(
    fit_intercept=False, 
    multi_class='multinomial', 
    solver ='saga'
).fit(X_train,y_train)
predictors = ['Neutral', 'Phase1', 'Phase2', 'Phase3', 'Phase4', 'Phase5', 'Phase6', 'Phase7', 'Phase8', 'NINO_34']
WT = [1, 2, 3, 4, 5, 6]
coefficients = xr.DataArray(
    data = fit.coef_,
    coords = {'Predictor': predictors, 'WT': WT},
    dims = ('WT', "Predictor")
)
coefficients.to_pandas()

In [ ]:
X_train = mrg[['nino_34', 'RMM1', 'RMM2']].values
fit = LogisticRegression(
    fit_intercept=True, 
    multi_class='multinomial', 
    solver ='saga'
).fit(X_train,y_train)
predictors = ['Intercept', 'NINO_34', 'RMM1', 'RMM2']
WT = [1, 2, 3, 4, 5, 6]
coefficients = xr.DataArray(
    data = np.append(fit.intercept_[:, np.newaxis], fit.coef_, axis=1),
    coords = {'Predictor': predictors, 'WT': WT},
    dims = ('WT', "Predictor")
)
coefficients.to_pandas()